In [ ]:
# default_exp core

# pandas temporal query language





In [0]:
#export

import re
import numpy as np
import pandas as pd


import ast
import glob
import ntpath
import os

from itertools import zip_longest, chain

from itertools import product
from functools import lru_cache
from functools import singledispatch

# General helper functions

## Info

In [0]:
#export
class Info():
  """
  A class to store information about the data and results from analysis
  """
  def __init__(self):
      self.evaluated = {}

## memory

In [0]:
#export
def memory(info, func, expr):
  """
  checks if the function has been called with the same argument previously and
  if so, returns the same results instead of running the function again

  args:
    - 
  """
  rows=None
  if info:
    if func in info.evaluated:
      if expr in info.evaluated[func]:
        rows = info.evaluated[func][expr]
    else:
      info.evaluated[func] = {}
  else:
    info = Info()
    info.evaluated[func] = {}
  return info, rows

## listify

In [0]:
#export
def listify(string_or_list):
    """
    return a list if the input is a string, if not: returns the input as it was

    Args:
        string_or_list (str or any):

    Returns:
        A list if the input is a string, if not: returns the input as it was

    Note:
        - allows user to use a string as an argument instead of single lists
        - cols='icd10' is allowed instead of cols=['icd10']
        - cols='icd10' is transformed to cols=['icd10'] by this function

    """
    if isinstance(string_or_list, str):
        string_or_list = [string_or_list]
    return string_or_list

## unique

In [0]:
#export
# A function to identify all unique values in one or more columns 
# with one or multiple codes in each cell

def unique(df, cols=None, sep=None, all_str=True):
  """
  Lists unique values from one or more columns
  
  sep (str): separator if cells have multiple values
  all_str (bool): converts all values to strings
  
  unique(df=df, cols='inpatient', sep=',')
  """
  # if no column(s) are specified, find unique values in whole dataframe
  if cols==None:
    cols=list(df.columns)
  cols = listify(cols)

  # multiple values with separator in cells
  if sep:
    all_unique=set()

    for col in cols:
      new_unique = set(df[col].str.cat(sep=',').split(','))
      all_unique.update(new_unique)
  # single valued cells
  else:
    all_unique = pd.unique(df[cols].values.ravel('K'))
  
  # if need to make sure all elements are strings without surrounding spaces
  if all_str:
    all_unique=[str(value).strip() for value in all_unique]

  return all_unique

In [0]:
#unique(df=df, cols='codes', sep=',')

## del dot and zero

In [0]:
#export
def del_dot(code):
  if isinstance(code, str):
    return code.replace('.','')
  else:
    codes = [c.replace('.','') for c in code]
  return codes

def del_zero(code, left=True, right=False):
  if isinstance(codes, str):
    codes=[code]
  if left:
    codes = [c.lstrip('0') for c in code]
  if right:
    codes = [c.rstrip('0') for c in code]
  if isinstance(code, str):
    codes=codes[0]
  return codes

# Notations

## expand hyphen

In [0]:
#export
# function to expand a string like 'K51.2-K53.8' to a list of codes 

# Need regex to extract the number component of the input string

# The singledispach decorator enables us to have the same name, but use 
# different functions depending on the datatype of the first argument.
#
# In our case we want one function to deal with a single string input, and
# another to handle a list of strings. It could all be handled in a single 
# function using nested if, but singledispatch makes it less messy and more fun!


# Here is the main function, it is just the name and an error message if the 
# argument does not fit any of the inputs that wil be allowed

@singledispatch
def expand_hyphen(expr):
  """
  Expands codes expression(s) that have hyphens to list of all codes

  Args:
      code (str or list of str): String or list of strings to be expanded 
  
  Returns:
      List of strings
  
  Examples:
      expand_hyphen('C00*-C26*')
      expand_hyphen('b01.1*-b09.9*')
      expand_hyphen('n02.2-n02.7')
      expand_hyphen('c00*-c260')
      expand_hyphen('b01-b09')
      expand_hyphen('b001.1*-b009.9*')
      expand_hyphen(['b001.1*-b009.9*', 'c11-c15'])
  Note:
      Unequal number of decimals in start and end code is problematic.
      Example: C26.0-C27.11 will not work since the meaning is not obvious:
      Is the step size 0.01? In which case C27.1 will not be included, while 
      C27.10 will be (and traing zeros can be important in codes)
  """
  raise ValueError('The argument must be a string or a list')

# register the function to be used if the input is a string
@expand_hyphen.register(str)
def _(expr):
    # return immediately if nothing to expand
    if '-' not in expr:
      return [expr]

    lower, upper = expr.split('-')
    
    lower=lower.strip()

    # identify the numeric component of the code
    lower_str = re.search("\d*\.\d+|\d+", lower).group()
    upper_str = re.search("\d*\.\d+|\d+", upper).group()
    # note: what about european decimal notation?
    # also note: what if multiple groups K50.1J8.4-etc


    lower_num = int(lower_str.replace('.',''))
    upper_num = int(upper_str.replace('.','')) +1
    
    if upper_num<lower_num:
      raise ValueError('The start code cannot have a higher number than the end code')

    # remember length in case of leading zeros 
    length = len(lower_str)

    nums = range(lower_num, upper_num)

    # must use integers in a loop, not floats
    # which also means that we must multiply and divide to get decimal back
    # and take care of leading and trailing zeros that may disappear
    if '.' in lower_str:
      lower_decimals = len(lower_str.split('.')[1])
      upper_decimals = len(upper_str.split('.')[1])
      if lower_decimals==upper_decimals:
        multiplier = 10**lower_decimals
        codes = [lower.replace(lower_str, format(num /multiplier, f'.{lower_decimals}f').zfill(length)) for num in nums]
      # special case: allow k1.1-k1.123, but not k.1-k2.123 the last is ambigious: should it list k2.0 only 2.00?
      elif (lower_decimals<upper_decimals) & (upper_str.split('.')[0]==lower_str.split('.')[0]):
        from_decimal = int(lower_str.split('.')[1])
        to_decimal = int(upper_str.split('.')[1]) +1
        nums = range(from_decimal, to_decimal)
        decimal_str = '.'+lower.split('.')[1]
        codes = [lower.replace(decimal_str, '.'+str(num)) for num in nums]
      else:
        raise ValueError('The start code and the end code do not have the same number of decimals')
    else:
        codes = [lower.replace(lower_str, str(num).zfill(length)) for num in nums]
    return codes
 

# register the function to be used if if the input is a list of strings
@expand_hyphen.register(list)
def _(expr):
  extended = []
  for word in expr:
    extended.extend(expand_hyphen(word))
  return extended


## expand star

In [0]:
#export
# A function to expand a string with star notation (K50*) 
# to list of all codes starting with K50

@singledispatch
def expand_star(code, all_codes=None):
  """
  Expand expressions with star notation to a list of all values with the specified pattern
  
  Args:
    expr (str or list): Expression (or list of expressions) to be expanded
    all_codes (list) : A list of all codes

  Examples:
    expand_star('K50*', all_codes=icd9)
    expand_star('K*5', all_codes=icd9)
    expand_star('*5', all_codes=icd9)

  """
  raise ValueError('The argument must be a string or a list')

@expand_star.register(str)
def _(code, all_codes=None): 
  # return immediately if there is nothing to expand
  if '*' not in code:
    return [code]

  start_str, end_str = code.split('*')

  if start_str and end_str:
    codes = {code for code in all_codes if (code.startswith(start_str) & code.endswith(end_str))}

  if start_str:
    codes = {code for code in all_codes if code.startswith(start_str)}
  
  if end_str:
    codes = {code for code in all_codes if code.endswith(end_str)}

  return sorted(list(codes))

@expand_star.register(list)
def _(code, all_codes=None):
  
  expanded=[]
  for star_code in code:
    new_codes = expand_star(star_code, all_codes=all_codes)
    expanded.extend(new_codes)
  
  # uniqify in case some overlap
  expanded = list(set(expanded))

  return sorted(expanded)



## expand colon

In [0]:
#export
# function to get all codes in a list between the specified start and end code 
# Example: Get all codes between K40:L52

@singledispatch
def expand_colon(code, all_codes=None):
  raise ValueError('The argument must be a string or a list')

@expand_colon.register(str)
def _(code, all_codes=None):
  """
  Expand expressions with colon notation to a list of complete code names
  code (str or list): Expression (or list of expressions) to be expanded
  all_codes (list or array) : The list to slice from

  Examples
    K50:K52
    K50.5:K52.19
    A3.0:A9.3

  Note: This is different from hyphen and star notation because it can handle 
  different code lengths and different number of decimals 

  """
  if ':' not in code:
    return [code]
  
  startstr, endstr = code.split(':')
  
  # remove spaces
  startstr = startstr.strip()
  endstr =endstr.strip()

  # find start and end position
  startpos = all_codes.index(startstr)
  endpos = all_codes.index(endstr) + 1
  
  # slice list
  expanded = all_codes[startpos:endpos+1]

  return expanded


@expand_colon.register(list)
def _(code, all_codes=None, regex=False): 
  expanded=[]

  for cod in code:
    new_codes = expand_colon(cod, all_codes=all_codes)
    expanded.extend(new_codes)
  
  return expanded


## expand regex

In [0]:
#export
# Return all elements in a list that fits a regex pattern

@singledispatch
def expand_regex(code, all_codes):
  raise ValueError('The argument must be a string or a list of strings')

@expand_regex.register(str)
def _(code, all_codes=None):
  code_regex = re.compile(code)
  expanded = {code for code in all_codes if code_regex.match(code)}
  # uniqify
  expanded = list(set(expanded))
  return expanded

@expand_regex.register(list)
def _(code, all_codes):  
  expanded=[]

  for cod in code:
    new_codes = expand_regex(cod, all_codes=all_codes)
    expanded.extend(new_codes)
  
  # uniqify in case some overlap
  expanded = sorted(list(set(expanded)))

  return expanded

## expand code

In [0]:
#export
@singledispatch
def expand_code(code, all_codes=None, 
                hyphen=True, star=True, colon=True, regex=False, 
                drop_dot=False, drop_leading_zero=False,
                sort_unique=True):
  raise ValueError('The argument must be a string or a list of strings')

@expand_code.register(str)
def _(code, all_codes=None, 
      hyphen=True, star=True, colon=True, regex=False, 
      drop_dot=False, drop_leading_zero=False,
      sort_unique=True):
  #validating input
  if (not regex) and (':' in code) and (('-' in code) or ('*' in code)):
    raise ValueError('Notation using colon must start from and end in specific codes, not codes using star or hyphen')

  if regex:
    codes = expand_regex(code, all_codes=all_codes)
    return codes
  
  if drop_dot:
    code = del_dot(code)
  
  codes=[code]

  if hyphen:
    codes=expand_hyphen(code)
  if star:
    codes=expand_star(codes, all_codes=all_codes)
  if colon:
    codes=expand_colon(codes, all_codes=all_codes)

  if sort_unique:
    codes = sorted(list(set(codes)))

  return codes

@expand_code.register(list)
def _(code, all_codes=None, hyphen=True, star=True, colon=True, regex=False, 
      drop_dot=False, drop_leading_zero=False,
      sort_unique=True):
  
  expanded=[]

  for cod in code:
    new_codes = expand_code(cod, all_codes=all_codes, hyphen=hyphen, star=star, colon=colon, regex=regex, drop_dot=drop_dot, drop_leading_zero=drop_leading_zero)
    expanded.extend(new_codes)
  
  # uniqify in case some overlap
  expanded = list(set(expanded))

  return sorted(expanded)



## expand columns

In [0]:
#export
def expand_columns(expr, all_columns=None, df=None, star=True, 
                   hyphen=True, colon=True, regex=None, info=None):
    """
    Expand columns with special notation to their full column names

    """
    notations = '* - :'.split()
    # return immediately if not needed
    if not any(symbol in expr for symbol in notations):
      return [expr]

    # get a list of columns of it is only implicity defined by the df
    # warning: may depreciate this, require explicit all_columns
    if df & (not all_columns):
      all_columns=list(df.columns)

    if regex:
      cols = [col for col in all_columns if re.match(regex, expr)]
    else:
      if hyphen:
        cols = expand_hyphen(expr)
      if star:
        cols = expand_star(expr, all_codes=all_columns)
      if colon:
        cols = expand_colon(expr, all_codes=all_columns)

    return cols

# More helper functions


## get rows

In [0]:
#export
# mark rows that contain certain codes in one or more colums
def get_rows(df, codes, cols=None, sep=None, pid='pid', info=None, fix=True):
  """
  Make a boolean series that is true for all rows that contain the codes
  
  Args
    df (dataframe or series): The dataframe with codes
    codes (str, list, set, dict): codes to be counted
    cols (str or list): list of columns to search in
    sep (str): The symbol that seperates the codes if there are multiple codes in a cell
    pid (str): The name of the column with the personal identifier

  """

  # check if evaluated previously
  info, rows = memory(info=info, func = 'get_rows', expr=codes)
  if rows:
    return rows

  # check if codes and columns need to be expanded (needed if they use notation) 
  if fix:
    # do this when if cols exist, but if it does not ...
    cols = expand_columns(expr=cols, all_columns=list(df.columns), info=info)
    all_codes = sorted(unique(df=df, cols=cols, sep=sep))
    codes = expand_code(codes, all_codes=all_codes)

  # codes and cols should be lists
  codes = listify(codes)
  cols = listify(cols)
  
  # approach depends on whether we have multi-value cells or not
  # if sep exist, then have multi-value cells
  if sep:
    # have multi-valued cells
    # note: this assumes the sep is a regex word delimiter
    codes = [rf'\b{code}\b' for code in codes]
    codes_regex = '|'.join(codes)
    
    # starting point: no codes have been found
    # needed since otherwise the function might return None if no codes exist
    rows = pd.Series(False*len(df),index=df.index)

   # loop over all columns and mark when a code exist  
    for col in cols:
      rows=rows | df[col].str.contains(codes_regex, na=False)
  
  # if not multi valued cells
  else:
    mask = df[cols].isin(codes)
    rows = mask.any(axis=1)
  return rows

## make codes

In [0]:
#export
def make_codes(n, letters=26, numbers=100, seed=False):
  """
  Generate a dataframe with a column of random codes
  
  Args:
    letters (int): The number of different letters to use
    numbers (int): The number of different numbers to use

  Returns
    A dataframe with a column with one or more codes in the rows

  """
  # each code is assumed to consist of a letter and a number
  alphabet = list('abcdefghigjklmnopqrstuvwxyz')
  letters=alphabet[:letters+1]
  
  # make random numbers same if seed is specified
  if seed:
    np.random.seed(0)

  # determine the number of codes to be drawn for each event
  n_codes=np.random.negative_binomial(1, p=0.3, size=n)
  # avoid zero (all events have to have at least one code)
  n_codes=n_codes+1
  
  # for each event, randomly generate a the number of codes specified by n_codes
  codes=[]
  for i in n_codes:
      diag = [np.random.choice(letters).upper()+
              str(int(np.random.uniform(low=1, high=numbers))) 
              for num in range(i)]

      code_string=','.join(diag)
      codes.append(code_string)

  # create a dataframe based on the list   
  df=pd.DataFrame(codes)    
  df.columns=['code']
  
  return df

In [0]:
make_codes(10)

## make data

In [0]:
#export
def make_data(n, letters=26, numbers=100, seed=False):
  """
  Generate a dataframe with a column of random codes
  
  Args:
    letters (int): The number of different letters to use
    numbers (int): The number of different numbers to use

  Returns
    A dataframe with a column with one or more codes in the rows

  """
  pid = range(n)
  df_person=pd.DataFrame(index = pid)

  #female = np.random.binomial(1, 0.5, size =n)
  gender = np.random.choice(['male', 'female'], size=n)
  region = np.random.choice(['north', 'south', 'east', 'west'], size=n)
  birth_year = np.random.randint(1920, 1980, size=n)
  birth_month = np.random.randint(1,12, size=n)
  birth_day = np.random.randint(1,28, size=n) # ok, I know!
  events_per_year = np.random.poisson(1, size=n)
  years = 2020 - birth_year
  events = years * events_per_year 
  events = np.where(events==0,1,events) 
  events = events.astype(int)
  all_codes=[]
  codes = [all_codes.extend(make_codes(n=n, letters=letters, 
                                       numbers=numbers, 
                                       seed=seed)['code'].tolist()) 
          for n in events]

  days_alive = (2020 - birth_year) *365

  days_and_events = zip(days_alive.tolist(), events.tolist())
  all_days=[]
  days_after_birth = [all_days.extend(np.random.randint(0, max_day, size=n)) for max_day, n in days_and_events]
  pid_and_events = zip(list(pid), events.tolist())
  all_pids=[]
  pids = [all_pids.extend([p+1]*e) for p, e in pid_and_events]

  df_events = pd.DataFrame(index=all_pids)
  df_events['codes'] = all_codes
  df_events['days_after'] = all_days 

  #df_person['female'] = female
  df_person['gender'] = gender

  df_person['region'] = region
  df_person['year'] = birth_year
  df_person['month'] = birth_month
  df_person['day'] = birth_day
  df = df_events.merge(df_person, left_index=True, right_index=True)
  df['birth_date'] = pd.to_datetime(df[['year', 'month', 'day']])
  df['event_date'] = df['birth_date'] + pd.to_timedelta(df.days_after, unit='d')
  del df['month']
  del df['day']
  del df['days_after']
  df['pid'] = df.index
  df.index_name = 'pid_index'
  df=df[['pid', 'gender', 'birth_date', 'event_date', 'region', 'codes']]
  # include deaths too?
  return df




In [0]:
#df = make_data(n=1000)

In [0]:
#df

In [0]:
#count_person('max 2 L35')
#count_person('x before y')

# formatting an expression

In [0]:
#export
def format_expression(expr):
  """
  formats and expressions do it can be evaluated
  """
  original = expr.copy()
  # easier to parse and split when space only exist between siginificant words
  expr=_remove_space(expr)
  
  # insert external variables (maybe unnecessary?)
  expr=_insert_external(expr)
  
  # if multiple options are specified in the expression, 
  # make one expression for each alternative specification
  expr=_get_expressions(expr)

  return exprs


## remove_space

In [0]:
#export
def remove_space(expr):
  no_space_before = r'(\s)([<=>,])'
  no_space_after = r'([<=>,])(\s)'

  expr = re.sub(no_space_before, r'\2', expr)
  expr = re.sub(no_space_after, r'\1', expr)
  return expr

## get_expressions

In [0]:
#export
def get_expressions(expr):
  """
  Makes a list of all possible statements from an expression, all possible combination of expressions involving ?[x, y, z] that are in the expressions
  
  >>>expr = 'min ?[2,3,4] of (K50, K51) in icd inside ?[10, 20, 30] days before 4AB02 in ncmp'
  >>>get_options(expr)

  """
  original = expr

  alternatives = re.findall('\?(\[.*?\])', expr)
  alt_list = [ast.literal_eval(alternative) for alternative in alternatives]

  combinations = product(*alt_list)

  all_expressions = []
  for n, combination in enumerate(combinations):
      new_expr = original
      for i, value in enumerate(combination):
          new_expr = new_expr.replace('?' + alternatives[i], str(value), 1)
      all_expressions.extend([new_expr])

  return all_expressions




In [0]:
expr = 'min ?[2,3,4] of (K50, K51) in icd inside ?[10, 20, 30] days before 4AB02 in ncmp'
get_expressions(expr)

## insert_external

In [0]:
#export
def insert_external(expr):
  """
  Replaces variables prefixed with @ in the expression with the
  value of the variable from the global namespace

  Example:
      x=['4AB02', '4AB04', '4AB06']
      expr = '@x before 4AB02'
      insert_external(expr)
  """
  externals = [word.strip('@') for word in expr.split() if word.startswith('@')]
  for external in externals:
      tmp = globals()[external]
      expr = expr.replace(f'@{external} ', f'{tmp} ')
  return expr

In [0]:
  x_1=['4AB02', '4AB04', '4AB06']
  expr = '@x_1 before 4AB02'
  insert_external(expr)

## insert columns


In [0]:
#export
def insert_columns(expr, cols=None, all_cols=None, code2col_rules = None, info=None):
    """
    insert column names in expressions (in col ...)
    
    logic: all conditions that do not contain column names, should end with an in statement
    general approach: 
      - split on keywords to get each condition separately
      - next: if the condition is about a column (age>20) no need to do anything
      - if, not, check if it has an 'in', if not: insert one!
     
     todo/problem: row selectors? code2col rule (a function that you can pass in?), sniffing, dict option, or in info?

    expr = 'max 2 of 4AB02 before 4AB04'
    expr = 'max 2 of 4AB02 in x before 4AB04'

    expr = '5th of 5th' # the code is here also a keyword ... problem - maybe of as long as we keep the of keyword ... but more difficult when we do not, at least for automatic column labeling!

    expr = 'max 2 of 4AB0552 before 4AB04'

    expr = 'max 2 of 4AB02 in ncmp' # should include zero ?
    expr = 'min ?[1,2,3,4] of 4AB02 in ncmp'
    expr = 'max ?[1,2,3,4] of 4AB02 in ncmp' # should include zero ?
    expr = 'min 2 of days>4'
    expr = 'min 8 of days>6'
    expr = 'min 3 of 4AB02 in ncmp within 200 days'

    insert_cols(expr, rule=col_rules)
    """

    split_words = [' and ', ' or ', ' before ', ' after ', ' within ']

    for word in split_words:
      expr = expr.replace(word, f'@split@{word}')
    conditions = expr.split('@split@')
    
    all_conditions=[]
    for condition in conditions:
      words = condition.split()
      if re.match('[><=]', word):
        pass
      elif len(words)==1:
        condition=condition + f' in {cols}'
      elif ' in ' not in expr:
        #alternative: words[-2] != 'in' but what if multiple cols already with spaces, then problem

        condition=condition + f' in {cols}'
      all_conditions.append(condition)
    new_expr = " ".join(all_conditions)
    new_expr = new_expr.replace('@split@','')
    return new_expr

# break up nested expressions

## outer parenthesis
first whole parenthesis expression, from start to finish, including nested

In [0]:
#export
def outer_parenthesis(expr):
  """
  identifies the first  parenthesis expression
  (may have nested parentheses inside it)
  rerurns the content, or just the string if ther are no parentheses
  may consider returning NONE then (if no?)

  """
  start_pos=expr.find('(')
  if start_pos == -1:
      between = expr
  else:
      n=1
      pos=start_pos
      while n>0:
          pos=pos+1
          if expr[pos]=='(':
              n=n+1
          elif expr[pos]==')':
              n=n-1
          elif pos>len(expr):
              print('Error: Not same number of opening and closing parenthesis')
      between = expr[start_pos+1: pos]
  return between


In [0]:

expr="nothing"
expr="more(than nothing)"
expr="outside((nested) and (nested))"
expr="(x) and (y)"
expr = "(x or q) before (y and z)"
expr = "c1 before (y and z)"


outer_parenthesis(expr)


## first inner parenthesis expression

In [0]:
#export
def first_inner_parenthesis(expr):
  """
  iterates until it finds the (first) innermost expression surrounded by 
  parentheses and returns this expression
  """
  new_expr = expr 
  
  while '(' in new_expr:
      outer= outer_parenthesis(new_expr)
      new_expr = outer
      #first_inner_parenthesis(new_expr) # hmm check why this is here    
  return new_expr


In [0]:
 
expr="nothing"
expr="more(than nothing)"
expr="outside((nested) and (nested))"
expr="(x) and (y)"
expr = "(x or q) before (y and z)"
expr = "c1 before (y and z)"
expr = "(x and (a or b)) before (y and z)"

expr = "(x and (a or (b after c))) before (y and z)"
expr = "x and b and c"

first_inner_parenthesis(expr)


## break_up
breaks an neste expresssion into its component parts

returns a dictionary with the name and the expressions to be evaluated
the dictionary is ordered: evaluate in order (since next expression may depend on calculation of previous expression)

logic: recursively find the innnermost expresion, store it, substitute, repeat

  possible todo (well, never! explicit is better than implicit here): implicit 
  breakup based on priority rules (and, or, before etc,
  like multiplication addition atc have priority rules that allow implicit breakup)

In [0]:
#export
def break_up(expr):
  """
  breaks up an expression with nested parenthesis into sub expressions with 
  no parenthesis, with the innermost expressions first, that needs to be 
  calculated before doing the outer expression. also replaces the statement with
  a single symbol in the larger expression
  """

  p=0
  new_expr = expr
  expr_store={}

  while "(" in new_expr:
      inner = first_inner_parenthesis(new_expr)
      new_expr = new_expr.replace(f'({inner})', f'p{p}')
      expr_store[f'p{p}'] = inner
      p=p+1
  return new_expr, expr_store


In [0]:
expr="nothing"
expr="more(than nothing)"
expr="outside((nested) and (nested))"
expr="(x) and (y)"
expr = "(x or q) before (y and z)"
expr = "c1 before (y and z)"
expr = "(x and (a or b)) before (y and z)"

expr = "(x and (a or (b after c))) before (y and z)"
expr = "x and b and c"

expr="outside and ((nested) and (nested))"
break_up(expr)


# eval stuff

After formatting and fixing a query, we end up with a list of expressions to be evaluatd (eval_expr)

After breaking up a standard expressions, we have a dictionary of sub expressions (X before Y, min 2 of X and max 3 of Y) (eval_sub_expr)

To evaluate a sub-expression, we breake it up into atomic statements (min 2 of Y), evaluate these separately and then apply the transformations that are appropriate based on the type of sub-expression it is (and, before/after etc) (eval_atom)

To evaluate an atom we use eval_row selection, get_rows and eval_prefix.

Alternative language? Paragraph, sentence, statement, condition, compound, molecule


eval_expr

##eval expr

In [0]:
#export
def eval_expr(df, expr, cols=None, sep=None, out='series', info=None, fix=True):
  # check if evaluated previously
  if cols:
    name = expr + out + str(cols)
  else: 
    name = expr + out

  info, rows = memory(info, 'eval_expr', name)
  if rows:
    return rows
  
  if fix:
    expr = remove_space(expr)
    expr = insert_external(expr)
    expr = insert_columns(expr=expr, cols=cols, all_cols=list(df.columns))
    #print(expr)

  final_expr, sub_expressions = break_up(expr)

  for name, expr in sub_expressions.items():
    df[name] = eval_sub(df=df, expr=expr, cols=cols, sep=sep, info=info)
  result = eval_sub(df=df, expr=final_expr, cols=cols, sep=sep, info=info)

  # return boolean series person (default) or rows, or pids
  if out == 'series':
    result = result.any(level=0)
  elif out == 'pids':
    result = set(result.index[result])
  
  info.evaluated['eval_expr'][name] = result

  return result

## eval sub (expression)

In [0]:
#export
def eval_sub(df, expr, cols=None, sep=None, info=None, fix=True):
  # check if evaluated previously
  if cols:
    name = expr + str(cols)
  else: 
    name = expr 

  info, rows = memory(info, 'eval_sub', name)
  if rows:
    return rows

  splitwords = 'and or not within after before inside outside'.split()
  operators = '= > <'.split()
  
  # a simple existence expression
  if not any(word in expr for word in splitwords):
    print('simple')
    rows = eval_atom(df=df, expr=expr, sep=sep, info=info)
  
  # and/or epression (to do:allow multiple and or without parenthesis?)
  # shortcut before splitting in sub_expressions: if only and or and not before etc: then can simplify: just split on and or, substitute, evaluate separate, keep parenthesis and do an eval
  elif (' and ' in expr) or (' or ' in expr):
    print('and or')
    rows = eval_and_or(df=df, expr=expr, sep=sep, info = info)

  # a before/after expression
  elif any(word in expr for word in [' before ', ' after ', ' simultaneous ']):
    print('before after')
    rows = eval_before_after(df=df, condition=expr, sep=sep, info=info, fix=fix)
  
  # within, not within expression
  elif ' within ' in expr:
    rows = eval_within(df=df, expr=expr, sep=sep, info=info)

  # an inside expression
  elif (' inside ' in expr) or (' outside ' in expr):
    rows = eval_inside_outside(df=df, expr=expr, sep=sep, info=info)

  # store result for future
  info.evaluated['eval_sub'][name] = rows

  return rows
  

(1st g after s) > 20
1st g after s > 20

1st g after s
g after s


## eval atom

In [0]:
#export
# atoms: [prefix] condition[row_selector] [in columns]
def eval_atom(df, expr, cols=None, sep=None, info=None):
  # precautionary move!
  expr=expr.strip()
  
  # check if evaluated previously
  if cols:
    name = expr + str(cols)
  else: 
    name = expr

  info, rows = memory(info, 'eval_atom', name)
  if rows:
    return rows  

  # check if it has a row selector, execute if so
  if '[rows:' in expr:
    row_selection = eval_row_selection(df=df, expr=expr)
    df=df[row_selection]
    # delete the row selector after applying it
    before, after = expr.split('[rows:')
    expr = before + after[1:]
  
  # starting point
  prefix = None
  words = expr.split()
  operator = any(operator in expr for operator in list('=><'))
  function_call = '(' in expr

  # is the atom a code based atom? example K50.1
  # code based atoms must have in or cols
  if (cols) or ('in' in words):
    if 'in' in words:
      codes, cols = expr.split(' in ')
      cols=cols.strip()
    else:
      codes = expr    
    if len(words)>3:
      prefix, codes = codes.rsplit(' ',1)

    # handle multiple cols 
    # in icd0, icd1, icd3 or in [icd0, icd1, icd3]
    if ',' in cols:
      if cols.startswith('['):
        cols = cols[1:-1].split(',')
        cols=[col.strip() for col in cols]

    # deal with list of codes [K50, K51]
    if ',' in codes:
      codes=codes[1:-1].split(',')
      codes=[code.strip() for code in codes]
 
    #expand codes and cols?
    rows = get_rows(df=df, cols=cols, codes=codes, sep=sep, info=info)

  # a simple column based atom: example glucose>8
  elif (operator) and (not function_call):
    prefix, expr = expr.rsplit(' ',1)
    rows=df.eval(expr)

  # a function based atom: example glucose.cumsum()>100
  elif (operator) and (function_call):
    prefix, expr = expr.rsplit(' ',1)
    rows = pd.eval(f"df.groupby('pid').{expr}")
    #alternative? rows = df.groupby(pid).apply(eval, expr)?
  
  if prefix:
    rows = eval_prefix(prefix, rows)
  
  # store results for future
  info.evaluated['eval_atom'][name] = rows
  
  return rows


In [0]:
df=make_data(1000,letters=10, numbers=5)


In [0]:
df.head()

In [0]:
#df['prescription_code'] =df.codes.str.split(',', expand=True)[0]
#df['ddd']=np.random.randint(1,99, size=len(df))
#df.to_csv('/content/sample_prescriptions.csv')
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
eval_atom(df=df, expr='E2 in codes', sep=',')

## eval row selection

In [0]:
#export
def eval_row_selection(df, expr, sep=None, info=None):
  """
  example K50[rows:age>20]
  example min 2 K51[rows: after hip_surgery]
  df[surgery_rows]....
  min 2 K51 inside after hip surgery
  min 2 K51 after hip surgery
  after age>20?
  """
  
  # check if evaluated previously
  info, rows = memory(info, 'eval_row_selection', expr)
  if rows:
    return rows 

  row_query = expr.split('[rows:')[1].split[']'][0]
  
  statement = any(operator in expr for operator in list('=><'))
  temporal = (' days ' in expr) 
  positional = (' events ' in expr) or (' event ' in expr)
  #positional = (expr.startswith('inside ')) or (expr.startswith('outside '))

  # statement:'age>20'
  if statement:
    rows=df.eval(row_query)
  # positional: 'inside/outside 5 events before/after/around X'
  elif positional:
    rows = create_time_interval(df=df, expr=expr,cols=cols, info=info)
  # temporal: 'after 1st S4'
  else:
    rows = create_time_interval(df=df, expr=expr,cols=cols, info=info)

  info.evaluated['eval_row_selection'][expr] = rows

  return rows


## eval prefix

In [0]:
#export
def eval_prefix(prefix, rows):
  interval=True
  freq = ['min ', 'max ', 'exactly ']
  first_last = [' first ', ' last']
  ordinal = r'(-?\d+)(st|nd|rd|th)'  # re to find and split 3rd into 3 and rd etc

  rowscum = rows.groupby(level=0).cumsum()

  # freq condition: min 5 of 4A
  if any(word in prefix for word in freq):
    word, num = prefix.split()
    num = int(num)

    if 'min' in word:
      select = (rowscum >= num)
    elif 'max' in word:  # double check!
      n_max = rowscum.max(level=0)
      select = (n_max <= num)
    elif 'exactly' in word:
      select = (rowscum == num)

  # beteween frequency or between ordinals (1st and 3rd)
  # note: inclusive between
  elif 'between ' in prefix:
    word, lower,_, upper = prefix.split()
    # interval/positional between 4th and 8th
    if re.match(prefix, ordinal):
        lower=int(lower[:-2])
        upper=int(upper[:-2])
        if lower > 0:
            aboverows = (rowscum >= lower)
        else:
            aboverows = (lastrowscum >= abs(lower))

        if upper > 0:
            belowrows = (rowscum <= upper)
        else:
            belowrows = (lastrowscum <= abs(upper))

        select = (aboverows & belowrows)
    # frequency between: between 4 and 8 of 4AB02
    else:
      lower=int(lower)
      upper=int(upper)
      select = rowscum.between(lower, upper, inclusive=True)

  # first, last range conditions: first 5 of 4A
  elif any(word.strip() in prefix for word in first_last):  # regex is better
    word, num = prefix.split()
    if '%' not in num:
      num = int(num)
      if 'first' in word:
        select = (rowscum <= num)
      if 'last' in word:
        select = (rowscum >= num)


  # pct condition: first 10% of 4A
  elif '%' in prefix:
    n_max = rowscum.groupby(level=0).max()
    pct = float(num.split(r'%')[0]) / 100
    pid_num = n_max * pct

    # first 1% of two observations includes 1st obs
    pid_num[pid_num < 1] = 1

    if word == 'first':
      # hmm, generalproblem: drop if pid is missing ...
      select = (rowscum < pid_num)

    if word == 'last':
      select = (rowscum > pid_num)

  # percentile condition
  elif ' percentile ' in prefix:
    event_num = rows.groupby(level=0).cumcount()
    n_count = rowscum.groupby(level=0).size()

    num = float(num.split(r'%')[0]) / 100

    pid_num = n_count * num

    if word == 'first':
      rows = (pid_num < event_num)

    if word == 'last':
      rows = (pid_num > event_num)
  
  # positional condition:  5th of 4a, 3rd to 8th of 4A, (3rd, 4th, 5th) of 4A
  # also allows: 2nd last (or even -5th last)
  elif re.match(ordinal, prefix):
    pos_str = prefix.rsplit(' ',1)[0].strip('(').strip(')')
    pos_nums = re.findall(ordinal, pos_str)
    pos_nums = tuple([int(pos[0]) for pos in pos_nums])

    # if the conditions includes last, need reversed cumsum
    # example 2nd last
    if ' last ' in pos_str or '-' in pos_str:
      n_max = rowscum.groupby(level=0).max().add(1)
      # reversed event number (by id)
      lastrowscum = (rowscum - n_max).abs()
      last_flag = 1
    else:
      last_flag = 0

    # single position: 5th of 4A
    if len(pos_nums) == 1:
      interval = False
      print('single position')
      if last_flag:
        select = (lastrowscum == pos_nums)
      else:
        select = (rowscum == pos_nums)

    # from-to positions: 3rd to 8th of 4A, 1st to -3rd
    elif ' to ' in pos_str:
        lower, upper = pos_nums
        if lower > 0:
            aboverows = (rowscum >= lower)
        else:
            aboverows = (lastrowscum >= abs(lower))

        if upper > 0:
            belowrows = (rowscum <= upper)
        else:
            belowrows = (lastrowscum <= abs(upper))

        select = (aboverows & belowrows)

  # list of positions (3rd, 5th, 7th)
  elif prefix.startswith('('):
    pos_str = prefix.rsplit(' ',1)[0].strip().strip('(').strip(')')

    pos_re = ordinal.replace(' ', '')  # last condition may have ) i.e. 25th)

    pos_nums = re.findall(pos_re, pos_str)
    pos_nums = tuple([int(pos[0]) for pos in pos_nums])

    pos_num = [num for num in pos_nums if num > 0]
    neg_num = [num for num in pos_nums if num < 0]

    pos_select = rowscum.isin(pos_nums)
    neg_select = rowscum.isin(pos_nums)
    select = (pos_select | neg_select)

  if interval==True:
    return select
  else:
    return rows & select


  # so far, have marked interval of events for expressions with qualifications
  # (existence conditions are not intervals). example: First 5 of 4A, markes
  # all events in the interval between the 1st and 5th of 4A
  # if we only want to pick the 4A events in this intereval, we and it with
  # the boolena for 4A existence (row). But sometimes we want to keep and use
  # the interval. For instance when the qualifiers are used in before/after
  # statements if the evaluated expression should be returned as 'exact row',
  # 'interval row' or pid existence

 

  





testing prefix

In [0]:
index = np.random.randint(100, size=1000)
code = np.random.binomial(1, p=0.4, size=1000)
rows=pd.Series(code, index=index).sort_index()
rows

In [0]:
df=rows.to_frame()
df['evaluated_prefix'] =eval_prefix('2nd A', rows=rows)
df.head(30)

## eval and or not

In [0]:
#export
def eval_and_or(df, expr, sep=None, info=None):
    split_words = [' and ', ' or ']

    for word in split_words:
      expr = expr.replace(word, f'@split@{word}')
    conditions = expr.split('@split@')
   
    for n, condition in enumerate(conditions):
      atom = condition.replace('and','').replace('or','')
      name = c + str(n)
      df[name] = eval_atom(df=df, expr=atom, sep=sep, info=info)
      final_expr = final_expr + condition.replace(atom, name)
   
    rows = df.eval(final_expr)
    return rows

## eval before after

In [0]:
#export
def eval_before_after(df, condition, cols=None, sep=None, 
                       info=None, out='pid', fix=True):
    # check if evaluated previously
    if cols:
      name = condition + out + str(cols)
    else: 
      name = condition + out
  
    info, rows = memory(info, 'eval_before_after', name)
    if rows:
      return rows 

    # replace conditions so intervals/multiples becomes positional
    # example: before first 5 of 4A --> before 5th of 4A
    # background: first 5 is not satisfied until ALL five have passed, while some other conditions are
    # may introduce shortcuts for some easy/common evaluations later (like 4A before 4B, easier than 4st of 4A before 1st of 4B?)
    # before and after are also different, may exploit this to create shortcuts
    condition = re.sub(r'last (-?\d+)', r'-\1st', condition)  

    condition = re.sub(r'first (-?\d+)', r'\1st', condition)
    # todo: also fix percentile --> find position, not first 5 percent

    before_expr, after_expr = re.split(' before | after | simultaneously ', condition)
    print(f'{before_expr},{after_expr}')
    # shortcut if ' simultaneous ' in condition: ...just a row and

    # check if the left side of the before expression has been calculated
    #if before in info.before_has_happened:
    #    before_has_happened = info.before_has_happened[before]
    #else:
    before_has_happened = eval_atom(df=df, expr=before_expr, cols=cols, sep=sep, info=info).groupby(level=0).cumsum().astype(bool).astype(int)
    #    info.before_has_happened[before] = before_has_happened
  
    # check if the right side of the before  expression have been calculated
    #if after in info.after_has_happened:
    #    after_has_happened = info.after_has_happened[after]
    #else:
    after_has_happened = eval_atom(df=df, expr=after_expr, cols=cols, sep=sep, info=info).groupby(level=0).cumsum().astype(bool).astype(int)
    #    info.after_has_happened[after] = after_has_happened
    
    both_exist = (before_has_happened.any(level=0)) & (after_has_happened.any(level=0))
 
    if ' before ' in condition:
      is_it_before = (before_has_happened - after_has_happened).groupby(level=0).sum() 
      endrows = both_exist & (is_it_before > 0)
    elif ' after ' in condition:
      is_it_after = (after_has_happened - before_has_happened).groupby(level=0).sum() 
      endrows = both_exist & (is_it_after > 0)
    elif ' simultaneous ' in condition:
      difference = (before_has_happened - after_has_happened).groupby(level=0).sum()
      endrows = both_exist & (difference ==0)
    
    #info.evaluated['eval_before_after'][name] = endrows

    return endrows

    
      


## eval within

Examples:
    expr= '4AB02 within 100 days after 4AB04'
    expr= 'min 2 of 4AB02 within 100 days'
    expr= '4AB02 within 50 to 100 days before 4AB04'
    expr= '4AB02 within 50 to 100 days before 4AB04'

    # maybe use inside on some?

    expr= 'min 4 of 4AB02 in ncmp within 100 days'

    expr= 'min 2 of 4AB02 within last 100 days'

    expr= 'min 2 of 4AB02 within 100 days from end'

    expr= 'min 2 of 4AB02 within first 100 days'

    expr= 'between 2 and 5 of 4AB02 within first 100 days' # avoid and? well, just use format and replace it with to?


    expr= 'min 2 of 4AB02 within 100 days from beginning'

    expr= 'min 2 of 4AB02 within 1st of 4AB04 to 5th of 4AB04'
    expr= 'min 2 of 4AB02 within 1st of 4AB06 to 3rd of 4AB04'

    expr= 'min 2 of 4AB02 within first 20 of 4AB04'

    expr= '3rd of 4AB02 within first 20 of 4AB04'

    expr= 'min 2 of 4AB02 within 100 days from 5th of 4AB04'
    expr = '3 or more of 4ab within 100 days'
    wstart, wend

    expr= 'min 4 of 4AB02 in ncmp within 100 days'
    expr= "min 4 of ncmp=='4AB02' within 100 days"
    expr= "at least 4 of ncmp=='4AB02' within 100 days"
    expr= "more than 4 of ncmp=='4AB02' within 100 days" # best language
    expr= "less than 4 of ncmp=='4AB02' within 100 days" # best language
    expr= "between 4 and 7 of ncmp=='4AB02' within 100 days" # best language inclusive or exclusive between
    expr= "5 or more of ncmp=='4AB02' within 100 days" # best language
    expr= "from 4 to 7 of ncmp=='4AB02' within 100 days" # best language
    expr= " 4 to 7 events with 4AB02 within 100 days" # best language #events problem ... again format?
    expr= " from 4 to 7 events with 4AB02 within 100 days" # best language #events problem ... again format?

    expr= " at least 5 events with 4AB02 within 100 days" # best language #events problem ... again format?
    expr= " no more than 5 events with 4AB02 in ncmp within 100 days" # best language #events problem ... again format?

    expr= 'min 3 of days>3 within 100 days'

    expr= 'ddd[pharma=='pharmaz'].sum()>97 within 100 days'
    

In [0]:
#export
def eval_within(df, condition, cols=None, sep=None, date='event_date', info=None, out='pid', pid='pid'):
    """
    mark observations that satisfy the conditions in a within statement

    examples: 
      - expr = 'min 3 4AB within 100 days'
      - expr = '4AB02 in ncmp within 50 to 100 days before 4AB04 in ncmp'
    """
    # within 100 days after 4AB --> within 0 to 100 days after 4AB
    if ((' after ' in condition) or (' before ' in condition)) and not (' to ' in condition):
      condition=condition.replace(' within ', ' within 0 to ')

    left, right = condition.split(' within ')
    print(f'within {left}, {right}')
    

    # within x to y days (better: between x to y days)
    # expr='4AB02 in ncmp within 50 to 100 days before 4AB04 in ncmp'
    if re.match(r'\d+ to \d+ ', right):
        print(f'within x to y')
        lower, _, upper, unit, direction, *rightsingle = right.split()
        if direction == 'around':
          condition = condition.replace(' around ', ' after ')
          after_prows = eval_within(df=df, condition=condition, cols=cols, sep=sep, info=info)
          condition = condition.replace(' after ', ' before ')
          before_prows = eval_within(df=df, condition=condition, cols=cols, sep=sep, info=info)
          endprows = (after_prows) | (before_prows)
          return endprows

        rightsingle = " ".join(rightsingle)
        lower = int(lower)
        upper = int(upper)
        
        lrows = eval_atom(df=df, expr=left, cols=cols, sep=sep, info=info)
        rrows = eval_atom(df=df, expr=rightsingle, cols=cols, sep=sep, info=info)

        pid_change = ((df[pid] - df[pid].shift()) != 0)

        rdates = df[date].where(rrows == 1, np.datetime64('NaT'))

        # if not have a date assign one to avoid ffill from person above
        # risky?
        rdates[(pid_change & ~rrows)] = np.datetime64('2100-09-09')  
              
        if direction == 'after':
            rdates = rdates.fillna(method='ffill')  # hmmm must use groupby here? or can it be avoided? inseret a 999 when pid change and fill it with nan after ffill?

        elif direction == 'before':
            rdates = rdates.fillna(method='bfill')

        rdates = rdates.where(rdates != np.datetime64('2100-09-09'), np.datetime64('NaT'))

        # allow other time units, within 5 seconds etc
        if unit == 'days':
            delta = (df[date] - rdates) / np.timedelta64(1, 'D')
        else:
            # add s if it is not there? like 1 day, 1 second?
            delta = (df[date] - rdates)
            delta = getattr(delta.dt, unit)

        if direction == 'before':
            delta = delta.abs()
        within = (delta >= lower) & (delta <= upper)
        endrows = (lrows & within) # nb, frequency conditions not work here I think: min 3 x within 10 to 100 days before S
        cpid = endrows.any(level=0)
        

    # pure within statements have few elements to the right
    # example min 2 4AB within 100 days
    elif len(right.split()) < 3:
        print(f'within x days')
        if ' in ' in left:
            word, num, codes, _, cols = left.split()
            rows = get_rows(df=df, codes=codes, cols=cols, sep=sep, info=info)

        #  'sum(days)>15 within 100 days' or 'min 5 of ddd>200 within 100 days'
        #  expr='sum(days)>15 within 100 days'
        elif re.search('[>=<]', left):
            if 'sum(' in left:
                # may want to create smaller dataframe first, if possible? focus on only some variable, columns, rows?
                sub = df.set_index(date)  # assume column date exist, should also drop rows with no time
                col, operator = left.split(')')
                col = col.replace('sum(', '').strip(')')
                threshold, unit = right.split()
                if unit == 'days': unit = 'D'
                eval_text = f"(sub.groupby('pid')['{col}'].rolling('{threshold}{unit}').sum()){operator}"
                rows = pd.eval(eval_text, engine='python')
                cpid = rows.any(level=0)
                return cpid
            # 'min 5 ddd>200 within 100 days'
            else:
                word, num, codes = left.split()
                rows = df.eval(codes)  # so far no sumsum etc, only 4 events with sugar_level>10 within 100 days

        # code expression not quantity expression
        # example: min 3 G2 within 100 days
        else:
            word, num, codes = left.split()
            cols = cols
            rows = get_rows(df=df, codes=codes, cols=cols)

        threshold, unit = right.split()
        threshold = int(threshold)
        num = int(num)

        if word == 'max': num = num + 1

        # may need to use expand cols to get the cols (not use cols expression here if it starred)
        sub = df[date][rows].dropna().to_frame()
        sub.columns = ['date']
        sub['pid'] = sub.index
        sub['shifted_date'] = sub['date'].shift(-num)
        sub['shifted_pid'] = sub['pid'].shift(-num)
        sub['diff_pid'] = (sub['pid'] - sub['shifted_pid'])
        sub['same_pid'] = np.where(sub.diff_pid == 0, 1, 0)

        sub = sub[sub.same_pid == 1]
        # sub['shifted_date'] = sub['date'].groupby(level=0).shift(int(num))
        # sub['shifted_pid'] = sub['pid'].groupby(level=0).shift(int(num))

        # todo: allow for different units here, months, weeks, seconds etc
        sub['diff_days'] = (sub['shifted_date'] - sub['date']) / np.timedelta64(1, 'D')

        # sub[sub.same_pid == 1]['diff_days'].dropna()/np.datetime64(1, 'D')

        if word == 'min':
            endrows = (sub['diff_days'] <= threshold)
            cpid = endrows.any(level=0)

        elif word == 'max':
            # n = df.index.nunique()
            endrows = (sub['diff_days'] <= threshold)
            cpid = ~endrows.any(level=0)

        # revise max and exactly

        elif word == 'exactly':
            endrows = (sub['diff_days'] <= threshold)
            n_max = endrows.groupby(level=0).sum()
            endrows = n_max == threshold
            cpid = endrows.any(level=0)

    #        #todo (also need to change parsing then ...)
    #        elif word=='between':
    #            endrows=(sub['diff_days']<=threshold)
    #            n_max = endrows.groupby(level=0).sum()
    #            endrows = n_max == threshold
    #            cpid = endrows.any(level=0)

    return cpid


## eval inside outside

In [0]:
#export
def eval_inside_outside(df, condition, cols=None, sep=None, pid='pid', info=None):
  """
    mark observations that satisfy the conditions in an inside/outside statement
    
    'inside/within/outside 5 events days before/after/around x'

    examples: 
      - expr = 'X inside 4 events after Y'
      - expr = 'X inside 4 events after each Y'
      - expr = 'always X inside 4 events after each Y'
      - expr = 'always X inside 4 events after a Y'
      - expr = 'X inside 3 events around Y'
      - expr = 'X outside 5 events after Y'
      - expr = 'no X before last 5 events' - hmm this is before after, not inside?
      - expr = 'no X inside 5 events before Y'   

      - expr = 'min 3 4AB inside last 5 events' - special
      - expr = 'X inside 1st and 5th Y' (between is better?) -special
      - expr = 'X inside 2 events before and 8 events after Y' - special

      - expr = 'min 2 X inside 4 events after Y'
      - expr = 'X inside 4 events after min 3 Y'
      - expr = 'X inside 4 to 7 events after min 3 Y'
  """
  # some horrible parsing
  if ' not ' in expr:
    pre, negate, post = expr.partition(' not ', expr)
    post='not ' + post
  else:
    pre, post = re.split(' inside | outside ', expr)
    if ' inside ' in expr:
      post = 'inside ' + post
    else:
      post = 'outside ' + post
  
  # mark relevant rows
  inside_rows = create_position_interval(df=df, expr=post, sep=sep, info=info)

  first_atom = eval_atom(df=df, expr=pre, sep=sep, info=info)

  endrows = inside_rows & first_atom
  # todo: warning, frequency conditions not work (at least not as expected)
  # todo: make a frequency version work? different keywords/sytax? groupby?
  return endrows
  




In [0]:
#expr='G4 in codes within 1 to 800 days around 3rd G2 in codes'
#a=eval_within(df=df, condition=expr, cols='codes', sep=',')
#df['a'] = a
#a.sum()

In [0]:
df.head()

# row selectors

Examples
- after/before s
- after 3rd 3s
- between 5th and 6th s
- within last 5 events
- within 100 days after s
- within 100 days after 2nd s
- within 50 days around 3rd s
- after min 3 s
- after glucose
- (pharma x after s) and (pharma y before pharma z)
- g after 1st s >20
- x before y after 2nd s
- x before (y after 2nd s)
- (x before y) after 2nd s
- after 2nd s: x before y
- x before (y before z)
- x before (y and z) and (y before z)
- (x before y) and (y before z)
- (x before y) after z
- x before y before z before q

before after statemetns have to be solved from right to left?








In [0]:
#def select_before_after(df, expr, sep=None, info=None):
#  word, atom = expr.split(' '.1)
#  rows = eval_atom(df=df, expr=expr. sep=sep, info=info)
  

In [0]:
#export
def eval_before_after(df, condition, cols=None, sep=None, 
                       info=None, out='pid', fix=True):
    # check if evaluated previously
    if cols:
      name = condition + out + str(cols)
    else: 
      name = condition + out
  
    info, rows = memory(info, 'row_eval', name)
    if rows:
      return rows 

    # replace conditions so intervals/multiples becomes positional
    # example: before first 5 of 4A --> before 5th of 4A
    # background: first 5 is not satisfied until ALL five have passed, while some other conditions are
    # may introduce shortcuts for some easy/common evaluations later (like 4A before 4B, easier than 4st of 4A before 1st of 4B?)
    # before and after are also different, may exploit this to create shortcuts
    condition = re.sub(r'last (-?\d+)', r'-\1st', condition)  

    condition = re.sub(r'first (-?\d+)', r'\1st', condition)
    # todo: also fix percentile --> find position, not first 5 percent

    before_expr, after_expr = re.split(' before | after | simultaneously ', condition)
    print(f'{before_expr},{after_expr}')
    # shortcut if ' simultaneous ' in condition: ...just a row and

    # check if the left side of the before expression has been calculated
    #if before in info.before_has_happened:
    #    before_has_happened = info.before_has_happened[before]
    #else:
    before_has_happened = eval_atom(df=df, expr=before_expr, cols=cols, sep=sep, info=info).groupby(level=0).cumsum().astype(bool).astype(int)
    #    info.before_has_happened[before] = before_has_happened
  
    # check if the right side of the before  expression have been calculated
    #if after in info.after_has_happened:
    #    after_has_happened = info.after_has_happened[after]
    #else:
    after_has_happened = eval_atom(df=df, expr=after_expr, cols=cols, sep=sep, info=info).groupby(level=0).cumsum().astype(bool).astype(int)
    #    info.after_has_happened[after] = after_has_happened
    
    both_exist = (before_has_happened.any(level=0)) & (after_has_happened.any(level=0))
 
    if ' before ' in condition:
      is_it_before = (before_has_happened - after_has_happened).groupby(level=0).sum() 
      endrows = both_exist & (is_it_before > 0)
    elif ' after ' in condition:
      is_it_after = (after_has_happened - before_has_happened).groupby(level=0).sum() 
      endrows = both_exist & (is_it_after > 0)
    elif ' simultaneous ' in condition:
      difference = (before_has_happened - after_has_happened).groupby(level=0).sum()
      endrows = both_exist & (difference ==0)
    
    info.evaluated['eval_before_after'][name] = endrows

    return endrows

    
      


## eval selector prefix

In [0]:
#export
def eval_selector_prefix(df, prefix, sep=None, cols=None, date='date', info=None):
  prefix=prefix.strip()
  words = prefix.split()
  
  # before last 5 events
  if ' event' in prefix:
    prefix = prefix.replace('events', 'event')
    #ad hoc, works with little code, but slow, optimize later
    df['event'] = 'e'
    prefix = prefix.replace('event', 'e in event')
  
  # just before
  if prefix=='before':
    print(prefix)
    rows=~(df['rowscum']>0)
  
  # just after
  elif prefix=='after':
    print(prefix)
    rows=df['rowscum']>0
  
  # 100 days before
  elif words[1]=='days':
    days, _, direction = prefix.split()
    rows=mark_days(df=df, max_days=days, direction=direction, date=date, info=info)

  # 3 events after
  elif (words[1]=='events') or (words[1]=='event'):
    days, _, direction = prefix.split()
    rows=mark_events(rows=df['atom_rows'], max_event=days, direction=direction, info=info)
  
  # 50 to 100 days after
  elif ('to' in words) and ('days' in words):            
    min_days, _, max_days, unit, direction = prefix.split()
    rows = mark_days(df=df, min_days=min_days, max_days=max_days, direction=direction) 
  
  # 2 to 5 events after
  elif ('to' in words) and (words[1]=='events') or (words[1]=='event'):            
    min_events, _, max_events, unit, direction = prefix.split()
    rows = mark_events(rows=df['atom_rows'], min_events=min_days, max_events=max_days, direction=direction) 
  
  return rows

## create time interval

In [0]:
#export
def create_time_intervals(df, expr, cols=None, sep=None, date='date', info=None):
  """
  expr='before 4AB04 in ncmp'
  expr='100 days before 4AB04 in ncmp'
  expr='50 to 100 days after 4AB04 in ncmp'
  
  expr='5 to 10 events after 4AB04 in ncmp'
  
  expr='50 days around 4AB04 in ncmp'
  expr='5 events around 4AB04 in ncmp' #next x events, inside 1 event after
  expr='before 3rd 4AB04 in ncmp'
  
  expr='100 days before last event'
  
  expr='between 3rd s in cod and 7th b in cod'
  
  expr='before age>20'
  expr='a pd statement' age >20
  expr='100 days before last event'

  expr='inside last 5 events'

  create_time_intervals(df=df, expr=expr)
  """
  original = expr
  words = expr.split()

  expr = re.sub(r'last (-?\d+)', r'-\1st', expr)  

  expr = re.sub(r'first (-?\d+)', r'\1st', expr)

  # todo: also fix percentile --> find position, not first 5 percent

  if any(word in words for word in 'before after around'.split()):
    splitted = re.split(r'\bbefore\b|\bafter\b|\bsimultaneously\b|\baround\b', expr)
    atom = splitted[-1]
    prefix=''
    for word in words:
      prefix = prefix + ' ' + word
      if word in 'before after sametime around'.split():
        break
    print(atom, prefix)
    atom_rows = eval_atom(df=df, expr=atom, cols=cols, sep=sep, info=info)
    rowscum = atom_rows.groupby(level=0).cumsum()
    df['atom_rows'] = atom_rows
    df['rowscum'] = rowscum
    rows = eval_selector_prefix(df=df, prefix=prefix, sep=sep, cols=cols, date=date, info=info)
  return rows


## mark days

In [0]:
#export
# mark days (ex 'within 22 days before/after x')
def mark_days(df, direction, max_days, min_days=0,inside=True, date='date', info=None):
  """
  mark days (ex 'inside/within/outside 22 days before/after/around x')
  """

  df['reference_event'] = np.where(df['atom_rows'] == 1, df[date], np.datetime64('NaT'))
  min_days=float(min_days)
  max_days=float(max_days)
  if direction=='around':
    rows_before = mark_days(df=df, max_days=max_days, direction='before', date=date, inside=True)
    rows_after = mark_days(df=df, max_days=max_days, direction='after', date=date, inside=True)
    rows = rows_before | rows_after
    if not inside: rows = ~rows
    return rows

  elif direction=='before':
    df['reference_event'] = df['reference_event'].groupby(level=0).fillna(method='bfill')  
  elif direction =='after':
    df['reference_event'] = df['reference_event'].groupby(level=0).fillna(method='ffill')

  event_diff = (df[date] - df['reference_event']).dt.days.abs() 
  
  # inside 50 to 100 days before x
  if inside:
    rows = event_diff.between(min_days, max_days)
  # outside 50 to 100 days before x
  else:
    rows = ~(event_diff.between(min_days, max_days))

  return rows

## create position interval

In [0]:
#export
# mark rows (ex 'within 5 events before/after x')
def create_position_interval(rows, expr, info=None):
  """
  mark events (ex '(not,always, never) inside/within/outside 5 events before/after/around x')
  """
  df = rows.to_frame()
  df.columns = ['reference_event']
  df['hard_way']=1
  df['event_num'] = df.groupby(level=0)['hard_way'].cumsum()

  pre, post = re.split(' inside | outside ', expr)
  # to do: handle pre if it exists ... maybe only handle not (never and always is higher level)
  
  last_atom = re.split(' before | after | around', post)[-1]
  
  inside_statement = expr.replace(pre,'').replace(last_atom,'')

  last_rows = eval_atom(df=df, expr=last_atom, cols=cols, sep=sep, info=info)

  inside_statement = post.replace(last_atom,'')

  # inside 2 to 5 events before x 
  # to do: validate since this should not work if the direction is around
  if ' to ' in inside_statement:
    in_or_out, min_events, _, max_events, direction =inside_statement.split()
  # inside 5 events before x 
  else:
    in_or_out, max_events, direction = inside_statement.split()
  
  if in_or_out in ('inside', 'between', 'not outside'):
    inside = True
  else:
    inside = False

  min_events=int(min_events)
  max_events=int(max_events)

  rows = mark_events(rows=last_rows, 
              direction=direction, 
              max_events=max_events, 
              min_events=min_events, 
              inside=inside, 
              info=info)

  return rows

## mark events


In [0]:
#export
# mark rows (ex 'within 5 events before/after')
def mark_events(rows, direction, max_events, min_events=0, inside=True, info=None):
  """
  mark events (ex '(not,always, never) inside/within/outside 5 events before/after/around x')
  """
  df = rows.to_frame()
  df.columns = ['reference_event']
  df['hard_way']=1
  df['event_num'] = df.groupby(level=0)['hard_way'].cumsum()


  min_events=int(min_events)
  max_events=int(max_events)

  if direction=='around':
    rows_before = mark_events(df=df, max_events=max_events, direction='before')
    rows_after = mark_events(df=df, max_events=max_events, direction='after')

    rows = rows_before | rows_after
    if not inside: rows = ~rows
    return rows

  elif direction=='before':
    df['reference_event'] = df['reference_event'].groupby(level=0).fillna(method='bfill')  
  elif direction =='after':
    df['reference_event'] = df['reference_event'].groupby(level=0).fillna(method='ffill')

  event_diff = (df['event_num'] - df['reference_event']).dt.days.abs() 
  
  # inside 50 to 100 events before x
  if inside:
    rows = event_diff.between(min_events, max_events)
  # outside 50 to 100 events before x
  else:
    rows = ~(event_diff.between(min_events, max_events))

  return rows

In [0]:
df=make_data(1000,letters=10, numbers=5)


In [0]:

df.head()

In [0]:
df['date'] = df.event_date

In [0]:
df = df.sort_values(['pid', 'date'])

In [0]:
df.head()

In [0]:
row = eval_atom(df=df, expr='B2 in codes', sep=',')
row.head()

In [0]:
eval_row_selection(df=df, expr='2 events before I2 in codes', sep=',')

In [0]:
expr='before 4AB04 in ncmp'
expr='100 days before 4AB04 in ncmp'
expr='50 to 100 days after 4AB04 in ncmp'
expr='before 3rd 4AB04 in ncmp'
expr='100 days before last event'
expr='after 1st D3 in codes'
expr='100 days before D3 in codes'
expr='500 days before H3 in codes'
expr='900 days around H3 in codes'
expr='100 to 600 days after H3 in codes' # interesting problem. is something is 150 days from a h3, but also 50 days from another h3?


In [0]:
df['around']=create_time_intervals(df=df, expr=expr, sep=',')
df.head(50)

In [0]:
df

In [0]:
df['atom_rows']=eval_atom(df=df, sep=',', expr='H4 in codes')
df['atom_rows']

In [0]:
df.groupby(level=0).atom_rows.cumsum()<1

In [0]:
create_time_intervals(df=df, expr=expr, sep=',')

In [0]:
expr='before D3 in codes'
x = re.split(r'\bbefore\b|\bafter\b|\bsimultaneously\b|\baround\b', expr)
x

In [0]:
df.head()

In [0]:
df=df.sort_values(['pid', 'event_date'])

df['date'] = df.event_date
df.head()

In [0]:
count_persons(df=df, expr='min ?[10, 20] G2 in codes', sep=',')

In [0]:
df.groupby('pid').size().value_counts().sort_index()


In [0]:
#export
def before_after(before, after, condition, info=None):

    before_has_happened = before.groupby(level=0).cumsum().astype(bool).astype(int)
    after_has_happened = after.groupby(level=0).cumsum().astype(bool).astype(int)

    both_exist = (before_has_happened.any(level=0)) & (after_has_happened.any(level=0))
 
    if condition=='before':
      is_it_before = (before_has_happened - after_has_happened).groupby(level=0).sum() 
      endrows = both_exist & (is_it_before > 0)
    elif condition=='after':
      is_it_after = (after_has_happened - before_has_happened).groupby(level=0).sum() 
      endrows = both_exist & (is_it_after > 0)
    elif condition == 'same time':
      difference = (before_has_happened - after_has_happened).groupby(level=0).sum()
      endrows = both_exist & (difference ==0)

    
    #expand to fit df length?
    endrows =endrows.reindex(index=before.index)
    #expand_endrows['result'] = endrows
    #endrows = expand_endrows['result']
    return endrows

In [0]:
index = np.random.randint(100, size=1000)
code1 = np.random.binomial(1, p=0.8, size=1000)
code2 = np.random.binomial(1, p=0.1, size=1000)

df=pd.DataFrame(code1, index=index).sort_index()
df['col2'] = code2
df.columns =['col1', 'col2']
df.head()

In [0]:
col1=df.col1
col2=df.col2
condition='before'
df['before'] = before_after(col1, col2, condition)
condition='after'
df['after'] = before_after(col1, col2, condition)
df.head(50)

In [0]:
df=make_data(1000,letters=10, numbers=5)
df.head()

# count persons

In [0]:
#export
def count_persons(df, expr, cols=None, sep=None, codebook=None, info=None, use_caching=True, insert_cols=True, fix=True):
    """

    count persons who satisfy the conditions in an expression

    examples
        expr = 'K50* in icd'
        expr = 'K50* before K51*'
        expr = '(K50 or K51) and K52'
        expr = 'min 3 of glucose>8 within 100 days'
        expr = '3rd of 4AB04 in ncmp before 3th of 4AB02 in ncmp'

    """
    expr = remove_space(expr)
    expr = insert_external(expr)
    exprs = get_expressions(expr)

    count = {}

    for expr in exprs:
      rows = eval_expr(df=df, expr=expr, cols=cols, sep=sep, info=info)
      count[expr] = rows.any(level=0).sum()

    # return only number if only one expression
    if len(count) == 1:
      return count[expr]

    return count

In [0]:
count_persons(df=df, expr='G3 in codes before I2 in codes', sep=',')

In [0]:
count_persons(df=df, expr='first 5 G3 before 3th G2', cols='codes', sep=',')

In [0]:
count_persons(df=df, expr='min 5 G3 before 3th G2', cols='codes', sep=',')

In [0]:
eval_atom(df=df, expr='1st G2 in codes', sep=True)

In [0]:
insert_columns('G2', cols='codes')

In [0]:
df.head()

In [0]:
count_persons(df=df, cols='codes', expr='B2 before H2', sep=',')

In [0]:
df.codes.str.contains('G2', na=False).any(level=0)

In [0]:
get_rows(df=df, codes='G2', cols='codes').any(level=0).sum()

In [0]:

    expr = "?['4AB02', '4AB04'] in ncmp"
    expr = '4AB02 in ncmp and 4AB04 in ncmp'
    expr = 'min 10 of 4AB02 in ncmp'
    expr = 'min ?[4,5,6] of 4AB02 in ncmp'
    expr =  'min 6 of 4AB02 in ncmp'
    expr =  'min 10 of 4AB02 in ncmp'

    expr = 'min ?[6,8] of 4AB02 in ncmp'
    expr = '1st of 4AB02 in ncmp'
    expr = '2nd of 4AB02 in ncmp'

    expr = '4AB02 in ncmp before 4AB04 in ncmp'
    expr = '4AB04 in ncmp before 4AB02 in ncmp'
    expr = '4AA23 in ncmp before 4AB02 in ncmp'
    expr = 'max 2 of 4AB02 in ncmp before 4AB04 in ncmp'
    expr = 'max 2 of 4AB02 in ncmp' # should include zero ?
    expr = 'min ?[1,2,3,4] of 4AB02 in ncmp'
    expr = 'max ?[1,2,3,4] of 4AB02 in ncmp' # should include zero ?
    expr = 'min 2 of days>4'
    expr = 'min 8 of days>6'
    expr = 'min 3 of 4AB02 in ncmp within 200 days'

    %time count_p(df=df, expr=expr, cols=None, codebook=None, info=None, sep=',')
    %time count_p(df=df, expr=expr, cols=None, codebook=None, info=info)

In [0]:
def count_persons(df, codes=None, cols=None, pid='pid', sep=None,
                  normalize=False, dropna=True, group=False, merge=False,
                  length=None, groupby=None, codebook=None, fix=True):
    """
    Counts number of individuals who are registered with given codes

    Allows counting across multiple columns and multiple codes in the same
    cells. For instance, there may be 10 diagnostic codes for one event (in
    separate columns) and in some of the columns there may be more than one
    diagnostic code (comma separated) and patient may have several such events
    in the dataframe.

    args:
        codes (str, list or dict): Codes to be counted. Star and hyphen
        notations are allowed. A dict can be used as input to merge codes
        into larger categories before counting. The key is the name of
        the category ('diabetes') and the value is a list of codes.

            Examples:
                codes="4ABA2"
                codes="4AB*"
                codes=['4AB2A', '4AB4A']
                codes = {'diabetes' = ['34r32f', '3a*']}

        cols (str or list): The column(s) with the codes. Star and colon
        notation allowed.
            Examples:
                cols = 'icdmain'
                cols = ['icdmain', 'icdside']
                # all columns starting with 'icd'
                cols = ['icd*'] # all columns starting with 'icd'
                # all columns including and between icd1 and icd10
                cols = ['icd1:icd10']

        pid (str): Column name of the personal identifier
        sep (str): The code seperator (if multiple codes in the same cells)
        normalize (bool, default: False): If True, converts to pct
        dropna (bool, default True): Include counts of how many did not get
            any of the specified codes
        length (int): If specified, will only use the number of characters
            from each code as specified by the length parameter (useful to
            count codes at different levels of granularity. For example,
            sometimes oe wants to look at how many people get detailed codes,
            other times the researcher wants to know only how many get general
            atc codes, say the first four characters of the atc)

    Examples
        >>> df.atc.count_persons(codes='4AB04')

        >>> df.atc.count_persons(codes='4AB04', dropna=False, normalize=True)

        >>> df.atc.count_persons(codes=['4AB*', '4AC*'])

        >>> df.atc.count_persons(codes=['4AB*', '4AC*'], group=True)
        >>> df.atc.count_persons(codes=['4AB*', '4AC*'], group=True, merge=True)

        >>> df.count_persons(codes={'adaliamumab':'4AB04'}, cols='ncmp', sep=',', pid='pid')

        >>> df.count_persons(codes='4AB04', cols='ncmp', groupby=['disease', 'cohort'])

        >>> df.groupby(['disease', 'cohort']).apply(count_persons, cols='ncmp', codes='4AB04', sep=',')

    """
    sub = df
    sub, cols = to_df(df=sub, cols=cols)
    cols = expand_cols(df=sub, cols=cols)
    if normalize:
        sum_persons = sub[pid].nunique()

    # if an expression instead of a codelist is used as input
    if isinstance(codes, str) and codes.count(' ') > 1:
        persons = use_expression(df=sub, expr=codes, cols=cols, sep=sep, out='persons', codebook=codebook, pid=pid)
        if normalize:
            counted = persons.sum() / len(persons)
        else:
            counted = persons.sum()


    # if codes is a codelist (not an expression)
    else:
        if fix:
            if not codes:
                counted = count_persons_all_codes(df=sub, cols=cols, pid=pid, sep=sep,
                                                   normalize=normalize, dropna=dropna, length=length, groupby=groupby)
                return counted
                # if some codes are specified, expand and format these, and reduce the df to the relevant codes
            else:
                # expands and formats columns and codes input
                codes, cols, allcodes, sep = fix_args(df=sub, codes=codes, cols=cols, sep=sep, group=group,
                                                       merge=merge)
                rows = get_rows(df=sub, codes=allcodes, cols=cols, sep=sep, fix=False)
                if not dropna:
                    sum_persons = df[pid].nunique()
                sub = sub[rows].set_index(pid,
                                          drop=False)  # unsure if this is necessary, may drop it. Requred if method on a series? well not as long as we add pid column and recreate a series as a df

        # make a df with the extracted codes
        code_df = extract_codes(df=sub, codes=codes, cols=cols, sep=sep, fix=False, series=False)

        labels = list(code_df.columns)

        counted = pd.Series(index=labels)

        # maybe delete groupby option, can be done outside df.groupby. apply ...
        if groupby:
            code_df = code_df.any(level=0)
            sub_plevel = sub.groupby(pid)[groupby].first()
            code_df = pd.concat([code_df, sub_plevel], axis=1)  # outer vs inner problem?

            code_df = code_df.set_index(groupby)
            counted = code_df.groupby(groupby).sum()

        else:
            for label in labels:
                counted[label] = code_df[code_df[label]].index.nunique()

        if not dropna:
            with_codes = code_df.any(axis=1).any(level=0).sum()  # surprisingly time consuming?
            nan_persons = persons - with_codes
            counted['NaN'] = nan_persons

        if normalize:
            counted = counted / sum_persons
        else:
            counted = counted.astype(int)

        if len(counted) == 1:
            counted = counted.values[0]

    return counted

In [0]:
eval_atom(df=df, expr='B1-B5 in codes', sep=',')

In [0]:
def eval_condition(expr):
  

In [0]:
@lru_cache()
def get_conditions(expr):
    split_on = [' or ', ' and ']
    split_rep = ' @split@ '
    for split_word in split_on:
        expr = expr.replace(split_word, split_rep)
    conditions = expr.split(split_rep)
    conditions = [condition.strip('(').strip(')') for condition in conditions]
    return conditions



In [0]:
expr = 'max 2 of 4AB0552 before 4AB04'
_insert_columns(expr, 'icd')

# evaluating atomic expressions


In [0]:
def eval_single(df, condition, cols=None, sep=None, codebook=None,
                out='pid', info=None):
    """
    evaluates a single expressions (1st 4A),
    not relational conditions (A before B, within 100 days after etc)

    condition ='first 5 of 4AB02 in ncmp'
    condition ='min 2 of days>10'
    condition ='ddd>10'
    condition ='ddd[4AB02 in codes]>10'
    condition ='ddd[4AB02 in codes].cumsum()>50'
    condition ='sum(ddd[4AB02 in codes])>50'

    a=eval_single(df=npr, condition=condition, sep=',')

    todo: info bank problems after allowing code selections?

    """
    # create temporary storage to avoid recalculations
    if not info: info = Info()

    original_condition = condition

    # no re-evaluation necessary if it it has been evaluated before
    if out == 'pid' and (condition in info.single_pid):
        return info.single_pid[condition]
    elif out == 'rows' and (condition in info.single_rows):
        return info.single_rows[condition]
    elif out == 'interval' and (condition in info.single_interval):
        return info.single_interval[condition]

    quantity = r'[>=<]'  # better to use term comparison
    freq = ['min ', 'max ', 'exactly ']
    first_last_between = [' first ', ' last ', ' between ']
    ordinal = r'(-?\d+)(st |nd |rd |th )'  # re to find and split 3rd into 3 and rd etc

    row_selection = ''

    # select sub df if specified by [] after a code
    if ('[' in condition) and (']' in condition):
        row_query = condition.split('[')[-1].split(']')[0]
        row_selection = row_query
        # check if evaluated before
        if row_query in info.single_rows:
            rows = info.single_rows[row_query]
        else:
            condition = condition.replace(f'[{row_query}]', '')

            if ' in ' in row_query:
                row_query = row_query.replace(' in ', ' in: ')  # using old use_expresssion wich requires in with colon

            relevant_rows = use_expression(df=df, cols=cols, expr=row_query, sep=sep)
            info.single_rows[row_query] = relevant_rows
        df = df[relevant_rows]

    # is it a functional expression? ddd.cumsum()>10
    # expr="ddd.cumsum()>10"
    # condition=expr
    # expr='gender.nunique()==1'
    # hmm what about properties like .is_monotonic? (no parenthesis!)
    # if ('.' in condition) and ('(' in condition) and (')' in condition):
    # still imperfect ... a code could also be a column name ... ok usually not also with a period mark in column name so ok
    if ('.' in condition) and (condition.split('.')[0] in df.columns):
        codetext = condition
        codes = re.split('[<=>]', condition)[0]

        if codes in info.single_rows:
            rows = info.single_rows[codes]
        # not evaluated before, so calc
        else:
            cols, funcexpr = condition.split('.')
            # a method
            if '(' in funcexpr:
                func, threshold = funcexpr.split(')')
                func, args = func.split('(')
                rows = pd.eval(f"tmpdf.groupby(['pid'])['{cols}'].transform('{func}', {args}) {threshold}",
                               engine='python')
            # an attribute (like is_monotonic)
            else:
                rows = pd.eval(f"tmpdf.groupby(['pid'])['{cols}'].transform(lambda x: x.{funcexpr})", engine='python')

            info.single_rows[codes] = rows

    # if it is a simple quantiative conditions (oxygen_level>20)
    elif re.search(quantity, condition):
        codetext = condition
        codes = condition.split()[-1]  # code condition always last hmm unnecessary
        # check if evaluated before
        if codes in info.single_rows:
            rows = info.single_rows[codes]
        # not evaluated before, so calc
        else:
            # sum(glucose_level)>10
            # if this, then may skip further processing?
            # well: 1st sum(glucose)>20 ok makes sense, maybe
            # but not: max 5 of sum(glucose)>20 ... well maybe
            # first 5 of sum(glucose)>20
            # if the modifiers does not make sense, the sum might be in the
            # list of other modifiers i.e. first 5, 3rd etc and not a
            # pre-modifier when finding rows (which allows skipping)

            # complex quantitative expression: sum(glucose_level)>10
            # better, more flexible ...: glucose.sum()>10 ... can make any function work, and can pass arguments

            if 'sum(' in codes:  # can use ddd.cumsum() now, keep this to double check
                col, operator = codes.split(')')
                col = col.replace('sum(', '').strip(')')
                eval_text = f"df.groupby(df.index)['{col}'].cumsum(){operator}"
                rows = pd.eval(eval_text, engine='python').fillna(False)  # is fillna false better than dropna here?
            # simple quantitative expression: glucose_level)>10
            else:
                rows = df.eval(codes).fillna(False)
            codecols = codes
            info.single_rows[codecols] = rows


    # code expression (involving a code, not a quantitative expressions
    else:
        codetext, incols = condition.split(' in ')
        codes = codetext.split()[-1].strip()  # codes always last in a simple string after cutting 'in cols'

        if incols.strip() == '':
            cols = cols
        else:
            cols = incols

        codecols = codes + ' in ' + cols + ' row ' + row_selection  # cannot use just codes to store rows since same code may be in different columns, so need to include col in name when storing

        # If conditions is about events in general, create an events column
        if (' event ' in codes) or (' events ' in codes):
            rows = pd.Series(True, index=df.index).fillna(False)
            codecols = ' event '
        # not a quantitative condition or an event conditions, so it is a code condition
        else:
            if codecols in info.rows:
                rows = info.rows[codecols]
            else:
                # cols = expand_cols(df=df, cols=cols)
                # expanded_codes = expand_codes(df=df, codes=codes, cols=cols, sep=sep)
                # allcodes=_get_allcodes(expanded_codes)
                # rows = get_rows(df=df, codes=allcodes, cols=cols, sep=sep, fix=False)
                rows = use_expression(df=df, expr=codes + ' in:' + cols, sep=sep)

                info.rows[codecols] = rows

    # is there a prefix to the conditions? if not, isolated condition, just return rows
    # if not, start preparing for calculating conditions with qualifiers
    # todo: quite messy! refactor: one function to evluate the code/expression itself, another to evalute the qualifier?
    if ' ' not in codetext.strip():
        # remember answer
        info.single_rows[codecols] = rows
        info.rows[codecols] = rows

        if out == 'pid':
            endrows = rows.groupby(level=0).any()
            info.single_pid[codecols] = endrows
            info.pid[codecols] = endrows
        else:
            endrows = rows
        return endrows

    # calculate and remember cumsum per person
    # use previous calculation if exist
    if codes in info.cumsum:
        rowscum = info.cumsum[codes]
    else:
        rowscum = rows.groupby(level=0).cumsum()
        info.cumsum[codecols] = rowscum

    ## if not a simple existence condition, it must be one of the conditions below

    # positional condition:  5th of 4a, 3rd to 8th of 4A, (3rd, 4th, 5th) of 4A
    # also allows: 2nd last (or even -5th last)
    if re.match(ordinal, codetext):
        pos_str = condition.split('of ')[0].strip().strip('(').strip(')')
        # pos_re = ordinal.replace(' ', '[ )]|') # last condition may have ) i.e. 25th)
        pos_re = ordinal.replace(' ', '')  # last condition may have ) i.e. 25th)

        pos_nums = re.findall(pos_re, pos_str)
        pos_nums = tuple([int(pos[0]) for pos in pos_nums])

        # if the conditions includes last, need reversed cumsum
        if ' last ' in pos_str or '-' in pos_str:
            n_max = rowscum.groupby(level=0).max().add(1)
            # reversed event number (by id)
            lastrowscum = (rowscum - n_max).abs()
            last_flag = 1
        else:
            last_flag = 0

        # single position: 5th of 4A
        if len(pos_nums) == 1:
            if last_flag:
                select = (lastrowscum == pos_nums)
            else:
                select = (rowscum == pos_nums)

        # from-to positions: 3rd to 8th of 4A, 1st to -3rd
        elif ' to ' in pos_str:
            lower, upper = pos_nums
            if lower > 0:
                aboverows = (rowscum >= lower)
            else:
                aboverows = (lastrowscum >= abs(lower))

            if upper > 0:
                belowrows = (rowscum <= upper)
            else:
                belowrows = (lastrowscum <= abs(upper))

            select = (aboverows & belowrows)

        # list of positions (3rd, 5th, 7th)
        elif pos_str.strip().startswith('('):
            pos_num = [num for num in pos_num if num > 0]
            neg_num = [num for num in pos_num if num < 0]

            if pos_num:
                pos_select = rowscum.isin(pos_nums)
            if neg_num:
                neg_select = rowscum.isin(pos_nums)
            select = (pos_select | neg_select)


    #  freq condition: min 5 of 4A
    elif any(word in codetext for word in freq):
        word, num, _, codes = codetext.split()
        num = int(num)

        if 'min' in word:
            select = (rowscum >= num)
        elif 'max' in word:  # doublecheck!
            n_max = rowscum.max(level=0)
            select = (n_max <= num)
        elif 'exactly' in word:
            select = (rowscum == num)


    # first, last range conditions: first 5 of 4A
    elif any(word.strip() in condition for word in first_last_between):  # regex is better
        word, num, _, codes = codetext.split()
        if '%' not in num:
            num = int(num)
            if 'first' in word:
                select = (rowscum <= num)
            if 'last' in word:
                select = (rowscum >= num)


    # if pct condition: first 10% of 4A
    elif '%' in codetext:
        n_max = rowscum.groupby(level=0).max()
        pct = float(num.split(r'%')[0]) / 100
        pid_num = n_max * pct

        # first 1% of two observations includes 1st obs
        pid_num[pid_num < 1] = 1

        if word == 'first':
            # hmm, generalproblem: drop if pid is missing ...
            select = (rowscum < pid_num)

        if word == 'last':
            select = (rowscum > pid_num)

    # percentile condition
    elif ' percentile ' in codetext:
        event_num = rows.groupby(level=0).cumcount()
        n_count = rowscum.groupby(level=0).size()

        num = float(num.split(r'%')[0]) / 100

        pid_num = n_count * num

        if word == 'first':
            rows = (pid_num < event_num)

        if word == 'last':
            rows = (pid_num > event_num)

    # so far, have marked interval of events for expressions with qualifications
    # (existence conditions are not intervals). example: First 5 of 4A, markes
    # all events in the interval between the 1st and 5th of 4A
    # if we only want to pick the 4A events in this intereval, we and it with
    # the boolena for 4A existence (row). But sometimes we want to keep and use
    # the interval. For instance when the qualifiers are used in before/after
    # statements if the evaluated expression should be returned as 'exact row',
    # 'interval row' or pid existence

    # store and return results
    if out == 'pid':
        endrows = (rows & select)
        endrows = endrows.any(level=0)
        info.single_pid[original_condition] = endrows
        info.single_rows[original_condition] = rows
    elif out == 'interval':
        endrows = select
        info.interval[original_condition] = endrows
    elif out == 'rows':
        endrows = (rows & select)
        info.single_rows[original_condition] = endrows

    return endrows


  
  



# get inpatient data

To test the functions and to calculate the Charslon index we need some data. Here we will use data on hospital visits from Medicare: 


In [0]:
# Use pandas
import pandas as pd

In [0]:
# Read synthetic medicare sample data on inpatient hospital stays
path = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/'
inpatient_file = 'DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip'

inpatient = pd.read_csv(path+inpatient_file)

inpatient.columns = inpatient.columns.str.lower()
# easier to use a column called 'pid' than 'desynpuf_id'
inpatient['pid']=inpatient['desynpuf_id']

#set index to the personal id, but also keep id as a column
inpatient = inpatient.set_index('pid', drop=False)
inpatient.index.name='pid_index'


In [0]:
# Have a look
inpatient.head()

In [0]:
# make a list of columns with information about diagnostic codes
icd_cols = list(inpatient.columns[inpatient.columns.str.startswith('icd9_dgns_cd')])
icd_cols

Make a list of all unique ICD9 codes that exist, a all_codes:

In [0]:
# Codes to calculate CCI using ICD-9 (CM, US, Enhanced)
# Source: http://mchp-appserv.cpe.umanitoba.ca/concept/Charlson%20Comorbidities%20-%20Coding%20Algorithms%20for%20ICD-9-CM%20and%20ICD-10.pdf

infarction = '''
      410* 
      412*
      '''

heart_failure = '''
        390.91 
        402.21 402.11 402.91 
        404.01 404.03 404.11 404.13 404.91 404.93 
        425.4-425.9 
        428*
        '''

peripheral_vascular = '''
        093.0
        437.3
        440*
        441*
        443.1-443.9
        447.1
        557.1 557.9
        V43.4
        '''

cerebrovascular = '''
        362.34
        430*-438*
        '''
dementia = '''
        290*
        294.1
        331.2
        '''

pulmonary ='''
      416.8 416.9
      490*-505* 
      506.4
      508.1 508.8
      '''
rheumatic = '''
      446.5
      710.0-710.4
      714.0-714.2 714.8
      725*
      '''

peptic_ulcer = '531*-534*'

liver_mild ='''
      070.22
      070.23
      070.32
      070.33
      070.44
      070.54
      070.6
      070.9
      570.*
      571.*
      573.3 573.4 573.8 573.9
      V42.7
      '''
# Interesting, diabetes seems to be 5 digits long in the data, but not the specified codes
diabetes_without_complication = '250.0*-250.3* 250.8* 250.9*'

diabetes_with_complication = '250.4*-250.7*'

plegia = '''
    334.1
    342.*
    343.*
    344.0-344.6
    344.9
    '''

renal = '''
    403.01 403.11,403.91 
    404.02 404.03 404.12 404.13 404.92 404.93
    582.*  
    583.0-583.7
    585*
    586*
    588.0
    V42.0
    V45.1
    V56*
    '''

malignancy = '''
    140*-172*
    174.0-195.8
    200*-208*
    238.6
    '''

liver_not_mild = '''
    456.0-456.2
    572.2-572.8
    '''

tumor = '196*-199*'

hiv = '042*-044*'

Put all the strings that describe the codes for the comorbitities in a single datastructure:

In [0]:
icd9 = unique(df=inpatient, cols = icd_cols, all_str=True)

In [0]:
# A dictionary with names of cormobitities and the associated medical codes

code_string = { 'infarction' : infarction, 
               'heart_failure' : heart_failure, 
               'peripheral_vascular' : peripheral_vascular, 
               'cerebrovascular' : cerebrovascular, 
               'dementia' : dementia, 
               'pulmonary' : pulmonary, 
               'rheumatic' : rheumatic, 
               'peptic_ulcer' : peptic_ulcer, 
               'liver_mild' : liver_mild, 
               'diabetes_without_complication' : diabetes_without_complication, 
               'diabetes_with_complication' : diabetes_with_complication, 
               'plegia' : plegia, 
               'renal' : renal, 
               'malignancy' : malignancy, 
               'liver_not_mild' : liver_not_mild, 
               'tumor' : tumor, 
               'hiv' : hiv}

Having created a all_codes, we can use the functions we have created to expand the description for all the different comorbidities to include all the specific codes:

In [0]:
codes = {disease : expand_code(codes.split(), 
                               all_codes=icd9,
                               drop_dot=True,
                               drop_leading_zero=True) 
        for disease, codes in code_string.items()}

And we can check if it really expanded the codes, for instance by examining the codes for mild liver disease:

In [0]:
codes['liver_mild']

In order to do the calculations, we need the weights associated with each comorbidity. These weights are related to the predictive power of the comorbididy for the probability of dying in a given time period. There are a few different standards, but with relatively minor varitions. Here we use the following:


In [0]:
charlson_points = { 'infarction': 1, 
                   'heart_failure': 1, 
                   'peripheral_vascular': 1, 
                   'cerebrovascular': 1, 
                   'dementia': 1, 
                   'pulmonary': 1, 
                   'rheumatic': 1, 
                   'peptic_ulcer': 1, 
                   'liver_mild': 1, 
                   'diabetes_without_complication': 1, 
                   'diabetes_with_complication': 2, 
                   'plegia': 2, 
                   'renal': 2, 
                   'malignancy': 2, 
                   'liver_not_mild': 3, 
                   'tumor': 6, 
                   'hiv': 6}


We also need the function that takes a set of codes and identifies the rows and persons who have the codes (a function we developed in a previous notebook):

In [ ]:
#hide
from nbdev.showdoc import *

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
Converted query_language.ipynb.
